In [1]:
# Check if the easydiffraction library is installed.
# If not, install it including the 'visualization' extras.
# This is needed, e.g., when running this as a notebook via Google Colab or
# Jupyter Notebook in an environment where the library is not pre-installed.
import builtins
import importlib.util

if hasattr(builtins, '__IPYTHON__'):
    if importlib.util.find_spec('easydiffraction') is None:
        !pip install 'easydiffraction[visualization]'


# Pair Distribution Function: Ni, NPD

This example demonstrates a pair distribution function (PDF) analysis of Ni,
based on data collected from a constant wavelength neutron powder diffraction
experiment.

The dataset is taken from:
https://github.com/diffpy/cmi_exchange/tree/main/cmi_scripts/fitNiPDF

## Import Library

In [2]:
import easydiffraction as ed

## Create Project

In [3]:
project = ed.Project()

## Set Plotting Engine

In [4]:
project.plotter.engine = 'plotly'


Current plotter changed to
plotly


## Add Sample Model

In [5]:
project.sample_models.add(name='ni')

In [6]:
project.sample_models['ni'].space_group.name_h_m = 'F m -3 m'
project.sample_models['ni'].space_group.it_coordinate_system_code = '1'
project.sample_models['ni'].cell.length_a = 3.52387
project.sample_models['ni'].atom_sites.add(
    label='Ni', type_symbol='Ni', fract_x=0.0, fract_y=0.0, fract_z=0.0, wyckoff_letter='a', b_iso=0.5
)

## Add Experiment

In [7]:
ed.download_from_repository('ni-q27r100-neutron_from-2.gr', destination='data')


⚠️ Warning
File 'data/ni-q27r100-neutron_from-2.gr' already exists and will not be overwritten.


In [8]:
project.experiments.add(
    name='pdf',
    sample_form='powder',
    beam_mode='constant wavelength',
    radiation_probe='neutron',
    scattering_type='total',
    data_path='data/ni-q27r100-neutron_from-2.gr',
)


Loading measured data from ASCII file
/Users/runner/work/diffraction-lib/diffraction-lib/tutorials/data/ni-q27r100-neutron_from-2.gr

Data loaded successfully
Experiment 🔬 'pdf'. Number of data points: 9801


In [9]:
project.experiments['pdf'].linked_phases.add(id='ni', scale=1.0)
project.experiments['pdf'].peak.damp_q = 0
project.experiments['pdf'].peak.broad_q = 0.03
project.experiments['pdf'].peak.cutoff_q = 27.0
project.experiments['pdf'].peak.sharp_delta_1 = 0.0
project.experiments['pdf'].peak.sharp_delta_2 = 2.0
project.experiments['pdf'].peak.damp_particle_diameter = 0

## Select Fitting Parameters

In [10]:
project.sample_models['ni'].cell.length_a.free = True
project.sample_models['ni'].atom_sites['Ni'].b_iso.free = True

In [11]:
project.experiments['pdf'].linked_phases['ni'].scale.free = True
project.experiments['pdf'].peak.broad_q.free = True
project.experiments['pdf'].peak.sharp_delta_2.free = True

## Run Fitting

In [12]:
project.analysis.current_calculator = 'pdffit'
project.analysis.fit()


Current calculator changed to
pdffit

Using experiment 🔬 'pdf' for 'single' fitting
🚀 Starting fit process with 'lmfit (leastsq)'...
📈 Goodness-of-fit (reduced χ²) change:


iteration,χ²,improvement [%]
1,2334.39,
9,416.28,82.2% ↓
15,210.14,49.5% ↓
21,207.13,1.4% ↓
45,207.10,


🏆 Best goodness-of-fit (reduced χ²) is 207.10 at iteration 44
✅ Fitting complete.



Fit results
✅ Success: True
⏱️ Fitting time: 52.58 seconds
📏 Goodness-of-fit (reduced χ²): 207.10
📏 R-factor (Rf): 9.83%
📏 R-factor squared (Rf²): 9.48%
📏 Weighted R-factor (wR): 9.48%
📈 Fitted parameters:


,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,ni,atom_sites,Ni,b_iso,0.5000,0.4281,0.0011,Å²,14.39 % ↓
2,ni,cell,,length_a,3.5239,3.5260,0.0000,Å,0.06 % ↑
3,pdf,linked_phases,ni,scale,1.0000,0.9892,0.0007,,1.08 % ↓
4,pdf,peak,,broad_q,0.0300,0.0220,0.0001,Å⁻²,26.77 % ↓
5,pdf,peak,,sharp_delta_2,2.0000,2.5587,0.0444,Å²,27.94 % ↑


## Plot Measured vs Calculated

In [13]:
project.plot_meas_vs_calc(expt_name='pdf', show_residual=True)